## Start

In [101]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
## Tags: #Tables #YandexFunctions

# Links:

[[citizens_st_mobile]]

[[subscriptions_st_mobile]]

[[entries_installation_points_dir_partner]]

[[installation_point_st_partner]]

[[citizen_payments_st_mobile]]

[[intercoms_st_partner]]

[[clichouse_schedule_function]]

_____

## t_full_citizen_id_in_flat_with_subscriptions

In [34]:
query_text = """--sql
    CREATE TABLE db1.t_full_citizen_id_in_flat_with_subscriptions
    (
        `report_date` Date,
        `address_uuid` String,
        `citizen_id` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY address_uuid
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [35]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_full_citizen_id_in_flat_with_subscriptions_mv
REFRESH EVERY 1 DAY OFFSET 4 HOUR 25 MINUTE TO db1.t_full_citizen_id_in_flat_with_subscriptions AS 
WITH t1 AS (SELECT 
	rmcicp.report_date AS report_date,
	flat_uuid
FROM db1.rep_mobile_citizens_id_city_partner AS rmcicp
JOIN db1.subscriptions_st_mobile_ch AS ssm 
	ON rmcicp.report_date = ssm.report_date 
	AND rmcicp.citizen_id = ssm.citizen_id
WHERE  ssm.state = 'activated'
)
SELECT
	t1.report_date AS report_date,
	rmcicp.address_uuid AS address_uuid,
	rmcicp.citizen_id AS citizen_id
FROM db1.rep_mobile_citizens_id_city_partner AS rmcicp
JOIN t1 AS ssm 
	ON rmcicp.report_date = ssm.report_date 
	AND rmcicp.flat_uuid = ssm.flat_uuid
WHERE t1.flat_uuid !=''
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [32]:
query_text = """
    DROP TABLE db1.t_full_citizen_id_in_flat_with_subscriptions
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,17207295,"""2c864d04-f138-4c51-90b0-a691cd…"


In [33]:
query_text = """
    DROP TABLE db1.t_full_citizen_id_in_flat_with_subscriptions_mv
    """
ch.query_run(query_text)


read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,111519862,"""74da82ee-7e41-4820-b81e-4f2beb…"


In [54]:
query_text = """
    SELECT
        *
    FROM db1.t_full_citizen_id_in_flat_with_subscriptions
    order by report_date desc
    limit 10
    """
ch.query_run(query_text)

report_date,address_uuid,citizen_id
date,str,i64
2025-08-26,"""00a54992-6b92-4d9e-b76f-f66499…",517593
2025-08-26,"""00a54992-6b92-4d9e-b76f-f66499…",515484
2025-08-26,"""00a54992-6b92-4d9e-b76f-f66499…",514924
2025-08-26,"""00a54992-6b92-4d9e-b76f-f66499…",514766
2025-08-26,"""00a54992-6b92-4d9e-b76f-f66499…",514759
2025-08-26,"""00a54992-6b92-4d9e-b76f-f66499…",514755
2025-08-26,"""00a54992-6b92-4d9e-b76f-f66499…",514753
2025-08-26,"""009c53bf-57e3-4111-bc1e-d3e117…",188116
2025-08-26,"""00a54992-6b92-4d9e-b76f-f66499…",514741


In [30]:
query_text = """
    SELECT
    report_date,
    count(*)
    FROM db1.t_full_citizen_id_in_flat_with_subscriptions_mv
    group by report_date
    order by report_date desc
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [7]:
query_text = """
SYSTEM REFRESH VIEW db1.t_full_citizen_id_in_flat_with_subscriptions_mv
"""

ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,2288725,"""f62694c3-685e-462c-9636-3cabe0…"


_____

## t_citizen_id_in_flat_with_subscriptions

In [57]:
query_text = """--sql
    CREATE TABLE db1.t_citizen_id_in_flat_with_subscriptions
    (
        `report_date` Date,
        `address_uuid` String,
        `citizen_id_in_flat_with_subscriptions` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY address_uuid
    """
ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

In [58]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_citizen_id_in_flat_with_subscriptions_mv
REFRESH EVERY 1 DAY OFFSET 5 HOUR 27 MINUTE TO db1.t_citizen_id_in_flat_with_subscriptions AS 
SELECT
    report_date,
    address_uuid,
    COUNT(DISTINCT citizen_id) as citizen_id_in_flat_with_subscriptions
FROM db1.t_full_citizen_id_in_flat_with_subscriptions
WHERE address_uuid !=''
GROUP BY report_date,
        address_uuid
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [60]:
query_text = """--sql
SELECT
    *
FROM db1.t_citizen_id_in_flat_with_subscriptions
ORDER BY report_date desc
LIMIT 10
    """
ch.query_run(query_text)

report_date,address_uuid,citizen_id_in_flat_with_subscriptions
date,str,i64
2025-08-26,"""015121f1-f869-430b-9da1-af7d35…",6
2025-08-26,"""0212b005-610a-4f8d-807b-d8d450…",2
2025-08-26,"""0387e2c1-5bc3-442a-9a56-0f1e66…",8
2025-08-26,"""01d27dc7-4ab1-428d-a0c1-c07c37…",2
2025-08-26,"""01a25aa5-a755-4b9c-830a-bd4a32…",86
2025-08-26,"""013c2dd5-b46e-49b4-aaf2-107be9…",5
2025-08-26,"""039e8a5c-12b6-42ee-a225-c34eec…",76
2025-08-26,"""01578474-3e1b-4a9d-bee8-bbe176…",16
2025-08-26,"""01836288-4230-49ce-9db6-c7b25c…",4


In [56]:
query_text = """
    DROP TABLE db1.t_citizen_id_in_flat_with_subscriptions_mv
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,1278476,"""957cde80-3414-4f47-b1fb-82fbb4…"


In [55]:
query_text = """
    DROP TABLE db1.t_citizen_id_in_flat_with_subscriptions
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,10756435,"""80bc9b6c-c472-40f0-ba30-408bac…"


____

## t_payments_amount

In [63]:
query_text = """--sql
    CREATE TABLE db1.t_payments_amount
    (
        `report_date` Date,
        `address_uuid` String,
        `payments_amount` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY address_uuid
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [64]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_payments_amount_mv
REFRESH EVERY 1 DAY OFFSET 5 HOUR 30 MINUTE TO db1.t_payments_amount AS 
SELECT
    report_date,
    address_uuid,
    sum(amount) AS payments_amount
FROM db1.rep_mobile_citizens_id_city_partner AS t_cit_id
JOIN db1.citizen_payments_st_mobile_ch AS citizen_payments_st_mobile
    ON citizen_payments_st_mobile.report_date = t_cit_id.report_date
    AND citizen_payments_st_mobile.citizen_id = t_cit_id.citizen_id
WHERE address_uuid != ''
GROUP BY report_date, address_uuid
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [102]:
query_text = """--sql
SELECT
    *
FROM db1.t_payments_amount
order by report_date desc
LIMIT 10
    """
ch.query_run(query_text)

report_date,address_uuid,payments_amount
date,str,i64
2025-08-27,"""0113915b-dc1d-4f5b-a56b-e83b20…",346
2025-08-27,"""01230ad8-53c1-43ee-8e45-d065a5…",69
2025-08-27,"""01b7ae17-dc42-4ea9-9bea-77db1c…",85
2025-08-27,"""0160c0f3-91c9-4e7d-8d94-b8731f…",138
2025-08-27,"""0163b402-1898-4079-9950-bdb3a5…",69
2025-08-27,"""016eefe2-6fbf-40d4-8549-12a68f…",207
2025-08-27,"""013286a5-ff2d-4b81-9a6d-59ef93…",69
2025-08-27,"""01836288-4230-49ce-9db6-c7b25c…",69
2025-08-27,"""01a25aa5-a755-4b9c-830a-bd4a32…",139


In [62]:
query_text = """
    DROP TABLE db1.t_payments_amount
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,2674779,"""0e681a8d-36df-4be4-8ab4-4cde90…"


In [61]:
query_text = """
    DROP TABLE db1.t_payments_amount_mv
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,1361897,"""3dd3a80c-6fc5-4cfb-a8eb-0177c9…"


____

## t_subscribed_citizen_id

In [80]:
query_text = """--sql
    CREATE TABLE db1.t_subscribed_citizen_id
    (
        `report_date` Date,
        `address_uuid` String,
        `subscribed_citizen_id` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY address_uuid
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [83]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_subscribed_citizen_id_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 33 MINUTE TO db1.t_subscribed_citizen_id AS 
    SELECT
        report_date,
        address_uuid,
        countDistinctIf(citizen_id, citizen_id != 0) AS subscribed_citizen_id
    FROM db1.subscriptions_report_citizens_flats_comerce_rep_mobile_total
    WHERE state = 'activated' AND address_uuid != ''
    GROUP BY report_date, address_uuid
    """
ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 57
 Code: 57. DB::Exception: Table db1.t_subscribed_citizen_id_mv already exists. (TABLE_ALREADY_EXISTS) (version 25.3.6.56 (official build))


In [84]:
query_text = """--sql
SELECT
    *
FROM db1.t_subscribed_citizen_id
order by report_date desc
LIMIT 10
    """
ch.query_run(query_text)

report_date,address_uuid,subscribed_citizen_id
date,str,i64
2025-08-25,"""01b1430e-2bc7-430e-b1fa-db14b5…",1
2025-08-25,"""02bc1ea2-fac1-4a11-9ccd-8faacf…",5
2025-08-25,"""01d856b1-a27c-4f87-a616-ac4945…",8
2025-08-25,"""01d9899e-f90f-46af-999e-c98775…",2
2025-08-25,"""02c8a97e-7555-48aa-acef-1bbe87…",4
2025-08-25,"""01ea7574-90df-48af-acf5-22e904…",1
2025-08-25,"""025cba1e-e224-45ea-9807-7a8e8a…",3
2025-08-25,"""01f36f31-666f-44b3-bd55-de1691…",2
2025-08-25,"""02c5c289-fb03-42c8-826a-fef34d…",3


In [78]:
query_text = """
    DROP TABLE db1.t_subscribed_citizen_id
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,3859964,"""56f4625e-c52b-4739-bfec-2143fc…"


In [79]:
query_text = """
    DROP TABLE db1.t_subscribed_citizen_id_mv
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,2468299,"""02a24012-eeee-4c91-9d1d-f7652a…"


____

## t_activated_citizen_id

In [87]:
query_text = """--sql
    CREATE TABLE db1.t_activated_citizen_id
    (
        `report_date` Date,
        `address_uuid` String,
        `activated_citizen_id` UInt64,
        `flat_uuid` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY address_uuid
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [88]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_activated_citizen_id_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 35 MINUTE TO db1.t_activated_citizen_id AS 
     SELECT
        report_date,
        address_uuid,
        countDistinctIf(citizen_id, citizen_id != 0) AS activated_citizen_id,	
        countDistinctIf(flat_uuid, flat_uuid != '') AS flat_uuid
    FROM db1.rep_mobile_citizens_id_city_partner
    WHERE address_uuid != ''
    GROUP BY report_date, address_uuid
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [89]:
query_text = """--sql
SELECT
    *
FROM db1.t_activated_citizen_id
ORDER BY report_date DESC
LIMIT 10
    """
ch.query_run(query_text)

report_date,address_uuid,activated_citizen_id,flat_uuid
date,str,i64,i64
2025-08-27,"""007687c9-d8df-46bc-99ba-da982b…",39,21
2025-08-27,"""0040d102-ccad-44b6-aff6-ecb20d…",49,25
2025-08-27,"""00532564-8275-468b-94eb-f34eea…",36,18
2025-08-27,"""0011f1e0-9e4d-411f-8f7d-5f9c55…",57,28
2025-08-27,"""002da97d-4439-4166-887a-ba29e0…",35,18
2025-08-27,"""008de4bb-cfde-4666-a470-fd32de…",7,3
2025-08-27,"""0056fbe5-cd13-42cc-85d3-470729…",27,20
2025-08-27,"""008e5977-504c-46f1-84d2-078959…",14,6
2025-08-27,"""0011aa18-5d4e-46d2-908e-7795ca…",82,48


In [85]:
query_text = """
    DROP TABLE db1.t_activated_citizen_id
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,18939536,"""9ec9895e-69e2-4db4-a9ce-5682d1…"


In [86]:
query_text = """
    DROP TABLE db1.t_activated_citizen_id_mv
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,6460724,"""c5570a39-e211-44aa-bd48-a2e179…"


____

In [ ]:
import pandas as pd

query_text = """
INSERT INTO db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
WITH t_citizen_id_in_flat_with_subscriptions AS(
    SELECT
        report_date,
        address_uuid,
        countIf(citizen_id != 0) AS citizen_id_in_flat_with_subscriptions
    FROM (
        SELECT 
            report_date,
            citizen_id,
            address_uuid,
            IF(ssm.state = 'activated', 1, 0) AS if_sub_active,
            max(if_sub_active) OVER (PARTITION BY flat_uuid, report_date ORDER BY report_date DESC) AS flat_with_sub_active
        FROM db1.rep_mobile_citizens_id_city_partner AS rmcicp
        LEFT JOIN db1.subscriptions_st_mobile_ch AS ssm 
            ON rmcicp.report_date = ssm.report_date 
            AND rmcicp.citizen_id = ssm.citizen_id
        WHERE report_date = toDate('{d}')
    ) AS x
    WHERE flat_with_sub_active = 1 AND address_uuid != ''
    GROUP BY report_date, address_uuid
),
t_payments_amount AS(
    SELECT
        report_date,
        address_uuid,
        sum(amount) AS payments_amount
    FROM db1.rep_mobile_citizens_id_city_partner AS t_cit_id
    JOIN db1.citizen_payments_st_mobile_ch AS citizen_payments_st_mobile
        ON citizen_payments_st_mobile.report_date = t_cit_id.report_date
        AND citizen_payments_st_mobile.citizen_id = t_cit_id.citizen_id
    WHERE report_date = toDate('{d}') AND address_uuid != ''
    GROUP BY report_date, address_uuid
),
t_subscribed_citizen_id AS(
    SELECT
        report_date,
        address_uuid,
        countDistinctIf(citizen_id, citizen_id != 0) AS subscribed_citizen_id
    FROM db1.subscriptions_report_citizens_flats_comerce_rep_mobile_total
    WHERE state = 'activated' AND address_uuid != '' AND report_date = toDate('{d}')
    GROUP BY report_date, address_uuid
),
t_activated_citizen_id AS (
    SELECT
        report_date,
        address_uuid,
        countDistinctIf(citizen_id, citizen_id != 0) AS activated_citizen_id,	
        countDistinctIf(flat_uuid, flat_uuid != '') AS flat_uuid
    FROM db1.rep_mobile_citizens_id_city_partner
    WHERE report_date = toDate('{d}') AND address_uuid != ''
    GROUP BY report_date, address_uuid
),
t1 AS (
    SELECT
        report_date,
        city,
        full_address,
        intercoms.partner_uuid AS partner_uuid,
        intercoms.installation_point_id AS installation_point_id,
        motherboard_ids,
        address_uuid,
        company_name, 
        partner_lk,
        tin
    FROM (
        SELECT
            report_date,
            installation_point_id,
            partner_uuid,
            arrayStringConcat(groupArray(motherboard_id), ',') AS motherboard_ids
        FROM db1.intercoms_st_partner_ch
        LEFT JOIN db1.intercoms_dir_asgard_ch 
            ON intercoms_st_partner_ch.intercom_uuid = intercoms_dir_asgard_ch.intercom_uuid
        WHERE report_date = toDate('{d}') AND installation_point_id != 0
        GROUP BY report_date, installation_point_id, partner_uuid
    ) AS intercoms
    LEFT JOIN db1.entries_installation_points_dir_partner_ch AS eipdp  
        ON intercoms.installation_point_id = eipdp.installation_point_id
    LEFT JOIN db1.companies_dir_partner AS cdp
        ON intercoms.partner_uuid = cdp.partner_uuid
)
SELECT
    t1.report_date AS report_date,
    city,
    full_address,
    partner_uuid,
    installation_point_id,
    motherboard_ids,
    t1.address_uuid AS address_uuid,
    company_name, 
    partner_lk,
    tin,
    activated_citizen_id,
    flat_uuid,
    subscribed_citizen_id,
    payments_amount,
    citizen_id_in_flat_with_subscriptions
FROM t1
LEFT JOIN t_citizen_id_in_flat_with_subscriptions 
    ON t1.report_date = t_citizen_id_in_flat_with_subscriptions.report_date 
    AND t1.address_uuid = t_citizen_id_in_flat_with_subscriptions.address_uuid
LEFT JOIN t_payments_amount 
    ON t1.report_date = t_payments_amount.report_date 
    AND t1.address_uuid = t_payments_amount.address_uuid
LEFT JOIN t_subscribed_citizen_id
    ON t1.report_date = t_subscribed_citizen_id.report_date 
    AND t1.address_uuid = t_subscribed_citizen_id.address_uuid
LEFT JOIN t_activated_citizen_id
    ON t1.report_date = t_activated_citizen_id.report_date 
    AND t1.address_uuid = t_activated_citizen_id.address_uuid
"""

start = '2023-10-01'
finish = '2025-08-19'
df = pd.date_range(start, finish)[::-1]

for ts in df:
    d = ts.strftime('%Y-%m-%d')
    ch.query_run(query_text.format(d=d))


In [92]:
query_text = """--sql
    CREATE TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_ch 
    (
        `report_date` Date,
        `city` String,
        `full_address` String,
        `partner_uuid` String,
        `installation_point_id` Int64,
        `motherboard_ids` String,
        `address_uuid` String,
        `company_name` String,
        `partner_lk` String,
        `tin` String,
        `activated_citizen_id` UInt64,
        `flat_uuid` UInt64,
        `subscribed_citizen_id` UInt64,
        `payments_amount` Int64,
        `citizen_id_in_flat_with_subscriptions` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY report_date
    """
ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

___
## t_subscribtions_citizens_by_companies_and_cities_address_mv

In [93]:
query_text = """--sql
CREATE MATERIALIZED VIEW db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
REFRESH EVERY 1 DAY OFFSET 5 HOUR 45 MINUTE TO db1.t_subscribtions_citizens_by_companies_and_cities_address_ch AS 
WITH t1 AS (
    SELECT
        report_date,
        city,
        full_address,
        intercoms.partner_uuid AS partner_uuid,
        intercoms.installation_point_id AS installation_point_id,
        motherboard_ids,
        address_uuid,
        company_name, 
        partner_lk,
        tin
    FROM (
        SELECT
            report_date,
            installation_point_id,
            partner_uuid,
            arrayStringConcat(groupArray(motherboard_id), ',') AS motherboard_ids
        FROM db1.intercoms_st_partner_ch
        LEFT JOIN db1.intercoms_dir_asgard_ch 
            ON intercoms_st_partner_ch.intercom_uuid = intercoms_dir_asgard_ch.intercom_uuid
        WHERE installation_point_id != 0
        GROUP BY report_date, installation_point_id, partner_uuid
    ) AS intercoms
    LEFT JOIN db1.entries_installation_points_dir_partner_ch AS eipdp  
        ON intercoms.installation_point_id = eipdp.installation_point_id
    LEFT JOIN db1.companies_dir_partner AS cdp
        ON intercoms.partner_uuid = cdp.partner_uuid
)
SELECT
    t1.report_date AS report_date,
    city,
    full_address,
    partner_uuid,
    installation_point_id,
    motherboard_ids,
    t1.address_uuid AS address_uuid,
    company_name, 
    partner_lk,
    tin,
    activated_citizen_id,
    flat_uuid,
    subscribed_citizen_id,
    payments_amount,
    citizen_id_in_flat_with_subscriptions
FROM t1
LEFT JOIN db1.t_citizen_id_in_flat_with_subscriptions 
    ON t1.report_date = t_citizen_id_in_flat_with_subscriptions.report_date 
    AND t1.address_uuid = t_citizen_id_in_flat_with_subscriptions.address_uuid
LEFT JOIN db1.t_payments_amount 
    ON t1.report_date = t_payments_amount.report_date 
    AND t1.address_uuid = t_payments_amount.address_uuid
LEFT JOIN db1.t_subscribed_citizen_id
    ON t1.report_date = t_subscribed_citizen_id.report_date 
    AND t1.address_uuid = t_subscribed_citizen_id.address_uuid
LEFT JOIN db1.t_activated_citizen_id
    ON t1.report_date = t_activated_citizen_id.report_date 
    AND t1.address_uuid = t_activated_citizen_id.address_uuid
	"""
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [103]:
query_text = """
    SELECT
        report_date,
        count(*)
    FROM db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
    group by report_date
    order by
    report_date desc
    LIMIT 10

    """
ch.query_run(query_text)

report_date,count()
date,i64
2025-08-27,32316
2025-08-26,32303
2025-08-25,32294
2025-08-24,32239
2025-08-23,32235
2025-08-22,32231
2025-08-21,32213
2025-08-20,32181
2025-08-19,32157


In [24]:
query_text = """
SYSTEM REFRESH VIEW db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
767420,"""45e98e73-ca6c-437f-82fc-3f1e20…"


In [90]:
query_text = """
    DROP TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_ch
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,103057759,"""1f72e2d0-08ad-43c8-8159-887cf2…"


In [91]:
query_text = """
    DROP TABLE db1.t_subscribtions_citizens_by_companies_and_cities_address_mv
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,3292557,"""d644f4f7-97fb-4cea-93f2-872532…"
